In [1]:
import cv2 
import time 
from cvzone.HandTrackingModule import HandDetector
from cvzone.ClassificationModule import Classifier

import numpy as np
import math

In [2]:
cap = cv2.VideoCapture(0)
detector = HandDetector(maxHands=1)
classifier = Classifier("C:\\Users\\dell\\Desktop\\SignLanguage_vs\\Model\\keras_model.h5" , "C:\\Users\\dell\\Desktop\\SignLanguage_vs\\Model\\labels.txt")
offset = 20
imgSize = 300
counter = 0

In [3]:
labels = ["Hello","I love you","No","Okay","Please","Thank you","Yes"]



In [4]:

while True:
    success, img = cap.read()
    if not success:
        print("Failed to capture image")
        break  # or continue, based on your use case

    imgOutput = img.copy()
    hands, img = detector.findHands(img)
    if hands:
        hand = hands[0]
        x, y, w, h = hand['bbox']

        # Ensure crop coordinates are within image bounds
        x_start, x_end = max(x - offset, 0), min(x + w + offset, img.shape[1])
        y_start, y_end = max(y - offset, 0), min(y + h + offset, img.shape[0])

        imgCrop = img[y_start:y_end, x_start:x_end]

        # Proceed only if imgCrop is valid
        if imgCrop.size > 0:
            aspectRatio = h / w

            if aspectRatio > 1:
                k = imgSize / h
                wCal = math.ceil(k * w)
                wGap = math.ceil((imgSize - wCal) / 2)
                imgResize = cv2.resize(imgCrop, (wCal, imgSize))
                imgWhite = np.ones((imgSize, imgSize, 3), np.uint8) * 255
                imgWhite[:, wGap:wCal + wGap] = imgResize
            else:
                k = imgSize / w
                hCal = math.ceil(k * h)
                hGap = math.ceil((imgSize - hCal) / 2)
                imgResize = cv2.resize(imgCrop, (imgSize, hCal))
                imgWhite = np.ones((imgSize, imgSize, 3), np.uint8) * 255
                imgWhite[hGap:hCal + hGap, :] = imgResize

            # Assuming getPrediction and labels are properly defined
            prediction, index = classifier.getPrediction(imgWhite, draw=False)

            cv2.rectangle(imgOutput, (x_start, y_start - 70), (x_start + 400, y_start - 10), (0, 255, 0), cv2.FILLED)
            cv2.putText(imgOutput, labels[index], (x, y - 30), cv2.FONT_HERSHEY_COMPLEX, 2, (0, 0, 0), 2)
            cv2.rectangle(imgOutput, (x_start, y_start), (x_end, y_end), (0, 255, 0), 4)

            cv2.imshow('ImageCrop', imgCrop)
            cv2.imshow('ImageWhite', imgWhite)

    cv2.imshow('Image', imgOutput)
    if cv2.waitKey(1) & 0xFF == ord('q'):  # Press 'q' to quit
        break

cap.release()
cv2.destroyAllWindows()



1/1 [==============================] - 0s 81ms/step
